In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from transform_output_format import get_4D_output, get_2D_output
from sklearn.base import clone
from sklearn.neural_network import MLPRegressor
from utils import load_data_input
import pickle
from sklearn.model_selection import train_test_split

In [100]:
def benchmark(sequence, sequence_cls):
    last_image = sequence[-1].copy()
    preds = []
    for i in range(4):
        preds.append(last_image*(sequence_cls[4+i]/sequence_cls[3]))
    return np.array(preds)[:,15:66,15:66]

In [101]:
GHI,CLS,SZA,SAA,dates = load_data_input("X_train_copernicus.npz")
y_train_csv = pd.read_csv('y_train_zRvpCeO_nQsYtKN.csv')
y_train_4D = get_4D_output(y_train_csv)

In [130]:
mlp_preds = pd.read_csv('mlp86.csv')
mlp_preds = get_4D_output(mlp_preds)

In [102]:
preds_benchmark = []

for ghi, cls in zip(GHI, CLS):
    preds_benchmark.append(benchmark(ghi, cls))

preds_benchmark = np.array(preds_benchmark)

In [103]:
GHI.shape

(1845, 4, 81, 81)

In [104]:
SZA[:,0:1,15:66,15:66].shape

(1845, 1, 51, 51)

In [174]:
X = np.concatenate([GHI[:,:,15:66,15:66],CLS[:,:,15:66,15:66]], axis=1)
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y_train_4D, test_size=0.1, random_state=69)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=52)

def prepare_data(sequence):
    """_summary_

    Args:
        sequence (array(nb_examples,nb_img,81,81)): _description_
    """
    nb_samples, nb_img, size1, size2 = sequence.shape
    seq_swap = sequence.swapaxes(1,2).swapaxes(2,3)
    return seq_swap.reshape((nb_samples*size1*size2,nb_img))


(1845, 12, 51, 51)


In [175]:
X_train_reshape = prepare_data(X_train)
y_train_reshape = prepare_data(y_train)
y_train_reshape.shape

(4317660, 4)

In [176]:
X_test_reshape = prepare_data(X_test)
y_test_reshape = prepare_data(y_test)

In [127]:
X_val_reshape = prepare_data(X_val)
y_val_reshape = prepare_data(y_val)

In [186]:
model= MLPRegressor(hidden_layer_sizes=(4,64,4),activation="relu", solver='adam', verbose=True, n_iter_no_change=10, learning_rate="invscaling", max_iter=200,)
model.fit(X_train_reshape, y_train_reshape)

Iteration 1, loss = 1160.10398388
Iteration 2, loss = 726.56125414
Iteration 3, loss = 723.58540087


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(hidden_layer_sizes=(4, 64, 4), learning_rate='invscaling',
             verbose=True)

In [ ]:
predictions = model.predict(prepare_data(X)).reshape(1845,51,51,4).swapaxes(2,3).swapaxes(1,2)

In [187]:
np.mean((model.predict(X_test_reshape)-y_test_reshape)**2)

1643.5158146844178

In [29]:
np.mean((model.predict(X_val_reshape)-y_val_reshape)**2)


NameError: name 'X_val_reshape' is not defined

In [173]:
GHI_t,CLS_t,SZA_t,SAA_t,dates_t = load_data_input("X_test_copernicus.npz")

In [85]:
preds_benchmark = []

for ghi, cls in zip(GHI_t, CLS_t):
    preds_benchmark.append(benchmark(ghi, cls))

preds_benchmark = np.array(preds_benchmark)

In [87]:
X = np.concatenate([GHI_t[:,:,15:66,15:66],CLS_t[:,:,15:66,15:66], preds_benchmark], axis=1)
X = prepare_data(X)


In [90]:
y_sub = model.predict(X)

In [91]:
y_sub.shape

(4788441, 4)

In [92]:
y_preds = y_sub.reshape(1841,51,51,4)
y_preds = y_preds.swapaxes(2,3).swapaxes(1,2)

In [93]:
y_preds.shape

(1841, 4, 51, 51)

In [94]:
y_preds_2D = get_2D_output(y_preds)
y_preds_2D.to_csv('mlp86.csv', index=False)